In [1]:
import numpy as np
import matplotlib.pylab as plt
import pandas as pd
from sklearn.neighbors import LSHForest
import numpy as np
from scipy.stats import mode
from sklearn import preprocessing
from sklearn.cross_validation import train_test_split
from sklearn.decomposition import RandomizedPCA
from annoy import AnnoyIndex

In [2]:
dataframe = pd.read_csv("transformed_WISDM.csv", header=None)
dataframe.head()
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
y_train = dataframe.ix[:, 44]
X_train = dataframe.drop(dataframe.columns[[0, 44]], axis=1)
x_numpy = X_train.values

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x_numpy)
dct = {"Jogging" : 1, "LyingDown" : 2, "Sitting" : 3, "Stairs" : 4, "Standing" : 5, "Walking" : 6}
labels = {1:"Jogging", 2:"LyingDown", 3:"Sitting", 4:"Stairs", 5:"Standing", 6:"Walking"}
y_train = y_train.replace(dct)
y_transformed = y_train.values

In [3]:
X_train, X_test, y_train, y_test = train_test_split(x_scaled, y_transformed, test_size=0.30, random_state=42)

In [4]:
pca = RandomizedPCA(n_components=34,whiten=True).fit(X_train)
x_pca = pca.transform(X_train)
x_test_pca = pca.transform(X_test)

In [5]:
f = x_pca.shape[1]
t = AnnoyIndex(f)

In [6]:
for i in xrange(x_pca.shape[0]):
    t.add_item(i,x_pca[i] )

In [7]:
t.build(200)

True

In [8]:
t.save('annoy.ann')

True

In [9]:
u = AnnoyIndex(f)
u.load('annoy.ann')

True

In [10]:
print y_test.shape

(1631,)


In [11]:

lst =[]
for i in xrange(x_pca.shape[0]):
    nn = u.get_nns_by_item(i, 10)
    votes, _ = mode(y_train[nn])
    lst.append(votes[0])


In [12]:
print len(lst)

3804


In [13]:
print y_train.shape

(3804,)


In [14]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, confusion_matrix
print classification_report(lst, y_train, target_names=[l for l in labels.values()])

             precision    recall  f1-score   support

    Jogging       0.68      0.93      0.79        60
  LyingDown       0.93      0.92      0.92       434
    Sitting       0.91      0.94      0.92       949
     Stairs       0.56      0.90      0.69       107
   Standing       0.92      0.87      0.89       620
    Walking       0.97      0.92      0.95      1634

avg / total       0.93      0.92      0.92      3804



In [15]:
conf_mat = confusion_matrix(lst, y_train)

fig = plt.figure(figsize=(6,6))
width = conf_mat.shape[1]
height = conf_mat.shape[0]

res = plt.imshow(conf_mat, cmap='summer', interpolation='nearest')
for i, row in enumerate(conf_mat):
    for j, c in enumerate(row):
        if c>0:
            plt.text(j-.2, i+.1, c, fontsize=16)
            
cb = fig.colorbar(res)
plt.title('Confusion Matrix')
_ = plt.xticks(range(6), [l for l in labels.values()], rotation=90)
_ = plt.yticks(range(6), [l for l in labels.values()])
plt.show()

In [16]:
print accuracy_score(y_train, lst)

0.9174553102


In [17]:
print confusion_matrix(y_test, lst)

ValueError: Found arrays with inconsistent numbers of samples: [1631 3804]